In [7]:
import pandas as pd
import numpy as np
import os
import warnings
import math
import scipy.interpolate
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm

In [8]:
# combined weather data
weather_path = os.path.join(os.getcwd(), 'data', 'weather_20190226.csv')
# coordinates of all the towns
town_coord_path = os.path.join(os.getcwd(), 'data', 'FieldSiteLocations.csv')

In [9]:
weather_path

'C:\\Users\\KZ26677\\Desktop\\JD project\\crop yield\\data\\weather_20190226.csv'

In [10]:
weather = pd.read_csv(weather_path, header = 0, parse_dates=[0])
# weather['LOCATION_TOWN'] = weather['LOCATION_TOWN'].str.upper()

In [11]:
weather.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN
0,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/1/2011,NaN,0.62,60.0,32.0
1,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/2/2011,NaN,0.00,38.0,18.0
2,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/3/2011,NaN,0.00,32.0,19.0
3,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/4/2011,NaN,0.00,43.0,23.0
4,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/5/2011,NaN,0.00,37.0,18.0


In [12]:
town_coord = pd.read_csv(town_coord_path, header = 0, parse_dates=[0])
town_coord.columns = ['City', 'Longitute-trimmed', 'Latitude-trimmed']
town_coord['City'] = town_coord['City'].str.upper()

In [13]:
weather.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,TMAX,TMIN
0,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/1/2011,NaN,0.62,60.0,32.0
1,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/2/2011,NaN,0.00,38.0,18.0
2,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/3/2011,NaN,0.00,32.0,19.0
3,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/4/2011,NaN,0.00,43.0,23.0
4,USC00113106,"FLORA, IL US",38.678,-88.4798,146.6,1/5/2011,NaN,0.00,37.0,18.0


In [14]:
town_coord.head()

,City,Longitute-trimmed,Latitude-trimmed
0,F_BELLEVILLE,-89.984,38.513
1,F_BETHANY,-88.741,39.645
2,F_CLAYTON,-90.953,40.028
3,F_DELAVAN,-89.547,40.372
4,F_DU QUOIN,-89.243,38.011


In [15]:
# latitude/longitude coordinates of all the stations
station_coord = weather.groupby(by=['STATION','LATITUDE','LONGITUDE'], as_index=False).first()[['STATION','LATITUDE','LONGITUDE']]
station_coord.head()

,STATION,LATITUDE,LONGITUDE
0,US1ILPT0002,40.04060,-88.70390
1,US1ILWD0008,41.88760,-89.81360
2,US1KSMP0014,38.39820,-97.52070
3,US1KYMM0003,38.06550,-83.98810
4,USC00110137,38.86702,-90.14886


In [16]:
EARTH_REDIUS = 6378.137

def rad(d):
    return d * math.pi / 180.0

def getDistance(lat1, lon1, lat2, lon2):
    """
    calculate the distance between two places
    """
    radLat1 = rad(lat1) # WA
    radLat2 = rad(lat2) # Wb
    radLon1 = rad(lon1) #Ja
    radLon2 = rad(lon2) #Jb
    a = radLat1 - radLat2
    b = radLon1 - radLon2
    # Haversine Formula
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2), 2) + 
                                math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s

In [17]:
# straight-line distance between Belleville and Mount Sterling Station based on getDistance function
print(getDistance(39.9841, -90.7525, 38.52, -89.98),'kilometers')

176.0598743614326 kilometers


In [18]:
# set the number of nearest stations
#num_nearest = 2

In [19]:
def nearest_stat(num_nearest):
    # a dictionary containing the nearest five stations' IDs and distances of each town
    nearest_stations = {}

    # a certain town
    for town_index in town_coord.index.values:
        # latitude/longitude of the town
        town_lon = town_coord.iloc[town_index]['Longitute-trimmed']
        town_lat = town_coord.iloc[town_index]['Latitude-trimmed']

        # a dictionary containing all the stations' IDs and distances to the town
        all_stations = {}

        # go throught all the stations and calculate the distance between the town and each station
        for station_index in station_coord.index.values:
            station_lon = station_coord.iloc[station_index]['LONGITUDE']
            station_lat = station_coord.iloc[station_index]['LATITUDE']
            # insert the distance between the town and a station
            all_stations[station_coord.iloc[station_index]['STATION']] = getDistance(town_lat, town_lon, station_lat, station_lon)

        # sort all the stations according to their distances to the town
        all_stations = sorted(all_stations.items(), key=lambda d: d[1])

        # get the five nearest stations
        nearest_stations[town_coord.iloc[town_index]['City']] = all_stations[:num_nearest]
    return nearest_stations
    
# get the nearest stations of Belleville
nearest_stations = nearest_stat(7)
#nearest_stations['BELLEVILLE']

In [20]:
nearest_stations['F_BELLEVILLE']

[('USW00013802', 12.49070191509044),
 ('USC00237452', 28.093192044681526),
 ('USC00112679', 33.03060160203802),
 ('USC00110137', 41.93192209565437),
 ('USC00116011', 57.71546612824573),
 ('USC00113693', 63.74815870243718),
 ('USC00116642', 85.4219683075872)]

In [21]:
def get_flag(param, station_id,flag_index = 2):
    """
    get a certain flag of a certain parameter's attribtue
    E.g. ',L,7,1700' for 'PRCP_ATTRIBUTE' indicates that the value should be removed because the 2nd field is populated
    """
    try:
        # split the attribute column by comma
        param_flags = weather[weather['STATION'] == station_id][param +'_ATTRIBUTES'].str.split(',', expand = True)
        # return the target flag column
        return param_flags[flag_index - 1]
    # the parameter does not have a corresponding attribute column
    except:
        # return a column filled with ''
        return pd.Series('', index = weather[weather['STATION'] == station_id].index)

In [22]:
sum(get_flag('TMAX', 'USW00013802') != '')

0

In [23]:
def IDW_interpolation(town, param, power,num_nearest, drop_invalid = False):
    nearest_stations = nearest_stat(num_nearest)
    station_id_list = [i[0] for i in nearest_stations[town]]
    distance_list = [i[1] for i in nearest_stations[town]]
    
    if drop_invalid == True:
        # get the date and target parameter columns of the first station
        combined_df = weather[weather['STATION'] == station_id_list[0]][['DATE', param]]
        # get the column of a certain attribute
        temp_flag = get_flag(param, station_id_list[0])
        # set certain values to NaN based on the attribute column
        combined_df.loc[temp_flag != '', param] = np.nan
        # append the other stations
        for station_id in station_id_list[1:]:
            temp_df = weather[weather['STATION'] == station_id][['DATE', param]]
            temp_flag = get_flag(param, station_id)
            temp_df.loc[temp_flag != '', param] = np.nan
            combined_df = pd.merge(combined_df, temp_df, on = 'DATE', how = 'outer')
        # rename the columns
        combined_df.columns = ['DATE'] + [param+str(i) for i in range(num_nearest)]
        combined_df = combined_df.drop_duplicates(keep = 'first')
    else:
        combined_df = weather[weather['STATION'] == station_id_list[0]][['DATE', param]]
        for station_id in station_id_list[1:]:
            temp_df = weather[weather['STATION'] == station_id][['DATE', param]]
            combined_df = pd.merge(combined_df, temp_df, on = 'DATE', how = 'outer')
        combined_df.columns = ['DATE'] + [param+str(i) for i in range(num_nearest)]
        combined_df = combined_df.drop_duplicates(keep = 'first')
    
    interpolated_value = []
    # go though all the rows
    for index, row in combined_df.iterrows():
        numerator = []
        denominator = []
        # a value of NaN should be left out during the calculation
        not_null_indice = []
        # indice of NaNs in a row
        for i in range(1, num_nearest+1):
            if row[i] == row[i]:
                not_null_indice.append(i)

        for i in not_null_indice:
            numerator.append(row[i]/(distance_list[i-1])**int(power))
            denominator.append(1/(distance_list[i-1])**int(power))
    
        try:
            interpolated_value.append(sum(numerator) / sum(denominator))
        # all values in the row are NaNs
        except:
            # set the interpolated value as NaN
            interpolated_value.append(np.nan)
    
    # add a column of interpolated values to the dataframe
    combined_df = combined_df.reset_index().join(pd.DataFrame(interpolated_value))
    del combined_df['index']
    combined_df.rename(columns = {0:'Interpolated '+param}, inplace=True)
    
    return combined_df

In [24]:
#CSV for Thomasboro
def write_interpolated_values(param,num_nearest, power):
    """
    write the interpolated values of a certain parameter into a csv
    """
    with open("THOMASBORO_"+param + "(p" + str(power) + 'n' + str(num_nearest) +").csv","w") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        
        writer.writerow(['CITY'] + list(IDW_interpolation(town_coord['City'][0], param, power, num_nearest).columns))

        with tqdm (total = len(town_coord['City'])) as pbar:
            for town in town_coord['City']:
                if town == 'THOMASBORO':
                    combined_df = IDW_interpolation(town, param, power,num_nearest)
                    for index, row in combined_df.iterrows():
                        writer.writerow([town] + [i for i in row])
                pbar.update()

In [25]:
nearest_stations_list = [1,2,3,5,7,9]

for x in nearest_stations_list:
    write_interpolated_values('TMAX',x,0.5)

for x in nearest_stations_list:
    write_interpolated_values('TMAX',x,1)

for x in nearest_stations_list:
    write_interpolated_values('TMAX',x,2)

for x in nearest_stations_list:
    write_interpolated_values('TMAX',x,3)

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 49766.30it/s]


In [26]:
nearest_stations_list = [1,2,3,5,7,9]

for x in nearest_stations_list:
    write_interpolated_values('PRCP',x,0.5)

for x in nearest_stations_list:
    write_interpolated_values('PRCP',x,1)

for x in nearest_stations_list:
    write_interpolated_values('PRCP',x,2)

for x in nearest_stations_list:
    write_interpolated_values('PRCP',x,3)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<?, ?it/s]


In [27]:
nearest_stations_list = [1,2,3,5,7,9]

for x in nearest_stations_list:
    write_interpolated_values('AWND',x,0.5)

for x in nearest_stations_list:
    write_interpolated_values('AWND',x,1)

for x in nearest_stations_list:
    write_interpolated_values('AWND',x,2)

for x in nearest_stations_list:
    write_interpolated_values('AWND',x,3)

100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 50901.75it/s]


In [28]:
not_dopped = IDW_interpolation('BELLEVILLE', 'TMAX', 2,2)
dropped = IDW_interpolation('BELLEVILLE', 'TMAX', 2,2, True)

KeyError: 'BELLEVILLE'

In [ ]:
not_dopped.equals(dropped)

In [ ]:
%%time
IDW_interpolation('BELLEVILLE', 'PRCP', 2,2).head(10)

In [ ]:
%%time
IDW_interpolation('BELLEVILLE', 'PRCP', 3).head(10)

In [ ]:
def lineplot(combined_df):
    """
    visualize the interpolated values for a certain parameter into a line plot
    """
    df = combined_df[['DATE', combined_df.columns[-1]]].set_index('DATE')
    df.index = pd.to_datetime(df.index)
    sns.set_context({"figure.figsize": (15, 8)})
    sns.set_style('darkgrid')
    ax = df.plot()
    plt.title('Line Plot of Interpolated Values')
    plt.xticks(rotation = 45)
    plt.show()

In [ ]:
lineplot(IDW_interpolation('BELLEVILLE', 'TMAX', 2))

In [ ]:
def boxplot(combined_df):
    """
    visualize the interpolated values for a certain parameter into a box plot
    """
    combined_df = combined_df.set_index('DATE')
    combined_df.index = pd.to_datetime(combined_df.index)
    combined_df['Year'] = combined_df.index.year
    
    sns.set_context({"figure.figsize": (15, 8)})
    ax = sns.boxplot(data = combined_df, x='Year',y= combined_df.columns[-2])
    plt.title('Box Plot of Interpolated Values')
    
    plt.show()

In [ ]:
boxplot(IDW_interpolation('BELLEVILLE', 'TMAX', 2))

In [ ]:
def write_interpolated_values(param, power = 3):
    """
    write the interpolated values of a certain parameter into a csv
    """
    with open(param + "(p" + str(power) + 'n' + str(num_nearest) +").csv","w") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        
        writer.writerow(['CITY'] + list(IDW_interpolation(town_coord['City'][0], param, power).columns))

        with tqdm (total = len(town_coord['City'])) as pbar:
            for town in town_coord['City']:
                combined_df = IDW_interpolation(town, param, power)
                for index, row in combined_df.iterrows():
                    writer.writerow([town] + [i for i in row])
                pbar.update()

In [ ]:
def write_interpolated_values_without_error_removal(param, power = 2):
    """
    write the interpolated values of a certain parameter into a csv
    """
    with open(param + "_(p" + str(power) + 'n' + str(num_nearest) +").csv","w") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        
        writer.writerow(['CITY'] + list(IDW_interpolation(town_coord['City'][0], param, power).columns))

        with tqdm (total = len(town_coord['City'])) as pbar:
            for town in town_coord['City']:
                combined_df = IDW_interpolation(town, param, power, True)
                for index, row in combined_df.iterrows():
                    writer.writerow([town] + [i for i in row])
                pbar.update()

In [ ]:
# p = 3 and n = 7


def write_interpolated_values_p3_n7(param,num_nearest, power):
    """
    write the interpolated values of a certain parameter into a csv
    """
    with open(param + "(p" + str(power) + 'n' + str(num_nearest) +").csv","w") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        
        writer.writerow(['CITY'] + list(IDW_interpolation(town_coord['City'][0], param, power, num_nearest).columns))

        with tqdm (total = len(town_coord['City'])) as pbar:
            for town in town_coord['City']:
                combined_df = IDW_interpolation(town, param, power,num_nearest)
                for index, row in combined_df.iterrows():
                    writer.writerow([town] + [i for i in row])
                pbar.update()

In [ ]:
town = town_coord['City'][2]
param = 'TMAX'
num = 7
num_nearest = 7
power = 3

station_id_list = [i[0] for i in nearest_stations[town]]
distance_list = [i[1] for i in nearest_stations[town]]
    

combined_df = weather[weather['STATION'] == station_id_list[0]][['DATE', param]]
for station_id in station_id_list[1:]:
    temp_df = weather[weather['STATION'] == station_id][['DATE', param]]
    combined_df = pd.merge(combined_df, temp_df, on = 'DATE', how = 'outer')
combined_df.columns = ['DATE'] + [param+str(i) for i in range(num_nearest)]
combined_df = combined_df.drop_duplicates(keep = 'first')

In [ ]:
station_id_list

In [ ]:
weather[weather['STATION'] == 'USW00093810'][['DATE', param]]

In [ ]:
IDW_interpolation(town_coord['City'][2], 'TMAX', 3, 7)

In [ ]:
write_interpolated_values_p3_n7('PRCP',7,3)

In [ ]:
write_interpolated_values_p3_n7('TMAX',7,3)
write_interpolated_values_p3_n7('AWND',7,3)

In [ ]:
write_interpolated_values_p3_n7('TMIN',7,3)